In [1]:
import numpy as np
import pandas as pd

csv = pd.read_csv("data/PID.csv",header= None)
data = csv.to_numpy()
shuffled_data = data.T[:, np.random.permutation(data.T.shape[1])].T ### Shuffle data
shuffled_data

array([[  3.   , 124.   ,  80.   , ...,   0.305,  26.   ,   0.   ],
       [  3.   , 158.   ,  70.   , ...,   0.344,  35.   ,   1.   ],
       [  5.   ,  95.   ,  72.   , ...,   0.37 ,  27.   ,   0.   ],
       ...,
       [  3.   , 113.   ,  50.   , ...,   0.626,  25.   ,   0.   ],
       [  0.   , 137.   ,  70.   , ...,   0.17 ,  22.   ,   0.   ],
       [  0.   , 141.   ,   0.   , ...,   0.205,  29.   ,   1.   ]])

In [2]:
r = shuffled_data.shape[0]
i = int(0.7*r)
training_data = np.array(shuffled_data[:i,:], dtype = np.float128)
testing_data = np.array(shuffled_data[i:,:], dtype = np.float128)

In [27]:
# Thêm thư viện
import numpy as np
import pandas as pd
# Hàm sigmoid
def sigmoid(x):
    return 1/(1+np.exp(-x))
# Đạo hàm hàm sigmoid
def sigmoid_derivative(x):
    return x*(1-x)
# Lớp neural network
class NeuralNetwork:
    def __init__(self, layers, alpha=0.005):
        #Mô hình layer ví dụ [2,2,1]
        self.layers = layers
        # Hệ số learning rate
        self.alpha = alpha
        # Tham số W, b
        self.W = []
        self.b = []
        # Khởi tạo các tham số ở mỗi layer
        for i in range(0, len(layers)-1):
            w_ = np.random.randn(layers[i], layers[i+1])
            b_ = np.zeros((layers[i+1], 1))
            self.W.append(w_/layers[i])
            self.b.append(b_)
    # Tóm tắt mô hình neural network
    def __repr__(self):
        return "Neural network [{}]".format("-".join(str(l) for l in self.layers))
    # Train mô hình với dữ liệu
    def out(self, X):
        out = X[:,:-1]
        for i in range(0, len(self.layers) - 1):
            out = sigmoid(np.dot(out, self.W[i]) + (self.b[i].T))
        return out
    def calculate_loss(self,X):
        predict = self.out(X)
        label = np.array(X[:,-1], dtype = np.float128)
        return -(np.sum(label*np.log(predict) + (1-label)*np.log(1-predict)))
    def fit_partial(self,X):
        x = np.array(X[:,:-1], dtype = np.float128)
        y = np.array(X[:,-1], dtype = np.float128)
        count = 0
        # quá trình feedforward
        while self.calculate_loss(X) > 10**-3 :
            A = [x]
            out = A[-1]
            for i in range(0, len(self.layers) - 1):
                out = sigmoid(np.dot(out, self.W[i]) + (self.b[i].T))
                A.append(out)
            # quá trình backpropagation
            y = y.reshape(-1, 1)
            dA = [-(y/A[-1] - (1-y)/(1-A[-1]))]
            dW = []
            db = []
            for i in reversed(range(0, len(self.layers)-1)):
                dw_ = np.dot((A[i]).T, dA[-1] * sigmoid_derivative(A[i+1]))
                db_ = (np.sum(dA[-1] * sigmoid_derivative(A[i+1]), 0)).reshape(-1,1)
                dA_ = np.dot(dA[-1] * sigmoid_derivative(A[i+1]), self.W[i].T)
                dW.append(dw_)
                db.append(db_)
                dA.append(dA_)
            # Đảo ngược dW, db
            dW = dW[::-1]
            db = db[::-1]
            # Gradient descent
            for i in range(0, len(self.layers)-1):
                self.W[i] = self.W[i] - self.alpha * dW[i]/X.shape[0]
                self.b[i] = self.b[i] - self.alpha * db[i]/X.shape[0]
            count = count + 1
            if count > 1000 :
                break

In [28]:
nn = NeuralNetwork([8,5,1])

In [29]:
nn.fit_partial(training_data)

In [30]:
nn.calculate_loss(training_data)

186088.82692925823513

In [8]:
nn.W

[array([[-0.4168446 , -0.01230711, -0.07912232,  0.04685681,  0.07609624],
        [-0.20182972, -0.25013562, -3.80699666,  0.19466943, -1.91126344],
        [ 0.03112794, -0.21807764, -0.54425086,  0.10713968, -1.02813508],
        [ 0.31406417,  0.0455551 , -0.1203092 , -0.13244833, -0.5212527 ],
        [-0.224492  , -0.15786027, -1.68969365,  0.08398449, -4.77049796],
        [-0.2917592 , -0.05819461, -0.82695822,  0.21663625, -0.29672625],
        [-0.14352876, -0.16248511, -0.13168416,  0.12836948,  0.15026227],
        [-0.86570841,  0.09107907, -0.69069356,  0.06283753, -0.82235586]],
       dtype=float128),
 array([[-1.06603368],
        [-0.29223698],
        [ 0.26086196],
        [-1.24815789],
        [ 0.71571195]], dtype=float128)]

In [12]:
import tkinter
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

matplotlib.use('TkAgg')


def forward(X, W_1, H, W_2):
    return X.dot(W_1).dot(W_2)


def loss(Y, Y_hat):
    return 1 / 2 * np.sum((Y - Y_hat) / Y.shape[0] * (Y - Y_hat))


def backward(X, H, y_hat, y, learning_rate):
    dy = (y_hat - y).reshape(1, -1)
    dw2 = H.T.dot(dy)
    dh = dw2.dot(dy.T)
    dw1 = X.reshape(1, -1).T.dot(dh.T)
    return dw2, dw1

if __name__ == '__main__':
    # Initialize
    N = 64
    D_in = 1000
    H = 100
    D_out = 10

    X = np.random.rand(N, D_in)
    Y = np.random.rand(N, D_out)

    # Init weights
    W_1 = np.random.rand(D_in, H)
    W_2 = np.random.rand(H, D_out)
    H = np.random.rand(1, H)

    # Store loss
    l = []
    learning_rate = 2e-12

    # Using batch gradient descent
    for i in range(50):
        Y_hat = forward(X, W_1, H, W_2)
        l.append(loss(Y_hat, Y))
        for j in range(N):
            dw2, dw1 = backward(X[j, :], H,
                                Y_hat[j, :], Y[j, :], learning_rate)
            W_2 -= learning_rate * dw2
            W_1 -= learning_rate * dw1

    plt.plot(l)
    plt.show()